## load library

In [2]:
#python

from __future__ import division

import os
import time
import yaml
import random

import json

from operator import itemgetter

from itertools import chain

from datetime import datetime

from typing import Any, Dict, Tuple, Union, List

import copy

#optuna

import optuna

#sympy
import sympy as sym

#numpy

import numpy as np

#scipy

from scipy.sparse.linalg import svds
from scipy.optimize import minimize_scalar, fsolve

#tensorly
import tensorly as tl
from tensorly.decomposition import parafac, partial_tucker

#pytorch

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import models
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder


#baseline

import warnings

warnings.filterwarnings('ignore')

import sys

sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('/opt/ml/code/src/'))))

import src

from src.dataloader import create_dataloader
from src.loss import CustomCriterion
from src.model import Model
from src.trainer import TorchTrainer
from src.utils.common import get_label_counts, read_yaml
from src.utils.macs import calc_macs
from src.utils.torch_utils import check_runtime, model_info
from src.augmentation.policies import simple_augment_test
from src.utils.inference_utils import run_model
from train import train


## define EVBMF function

In [7]:
def tau(x, alpha):
    return 0.5 * (x-(1+alpha) + np.sqrt((x-(1+alpha))**2 - 4*alpha))

In [9]:
def sym_phi(tau):
    return (np.log(np.sqrt(alpha)*tau+1.0)/((np.sqrt(alpha)*tau)))+(np.log(tau/np.sqrt(alpha)+1.0)/(tau/np.sqrt(alpha)))-1.0

In [44]:
def Simple_EVBMF(Y, sigma2=None, H=None):
    """Implementation of the analytical solution to Empirical Variational Bayes Matrix Factorization.
    This function can be used to calculate the analytical solution to empirical VBMF. 
    This is based on the paper and MatLab code by Nakajima et al.:
    "Global analytic solution of fully-observed variational Bayesian matrix factorization."
    Notes
    -----
        If sigma2 is unspecified, it is estimated by minimizing the free energy.
        If H is unspecified, it is set to the smallest of the sides of the input Y.
    Attributes
    ----------
    Y : numpy-array
        Input matrix that is to be factorized. Y has shape (L,M), where L<=M.
    
    sigma2 : int or None (default=None)
        Variance of the noise on Y.
        
    H : int or None (default = None)
        Maximum rank of the factorized matrices.
        
    Returns
    -------
    U : numpy-array
        Left-singular vectors. 
        
    S : numpy-array
        Diagonal matrix of singular values.
        
    V : numpy-array
        Right-singular vectors.
        
    post : dictionary
        Dictionary containing the computed posterior values.
        
        
    References
    ----------
    .. [1] Nakajima, Shinichi, et al. "Global analytic solution of fully-observed variational Bayesian matrix factorization." Journal of Machine Learning Research 14.Jan (2013): 1-37.
    
    .. [2] Nakajima, Shinichi, et al. "Perfect dimensionality recovery by variational Bayesian PCA." Advances in Neural Information Processing Systems. 2012.     
    """   
    L,M = Y.shape #has to be L<=M
    
    
    if L > M:
        Y = Y.T
        L,M = Y.shape
        
    if H is None:
        H = L
        
    global alpha

    alpha = L/M

    #tauubar = 2.5129*np.sqrt(alpha)
    
    #tauubar = 2.5129*alpha
    
    tauubar_initial = 2.5129*np.sqrt(alpha)
    tauubar = fsolve(sym_phi,tauubar_initial)
    
    
    #SVD of the input matrix, max rank of H
    U,s,V = np.linalg.svd(Y)
    U = U[:,:H]
    s = s[:H]
    V = V[:H].T

    #Calculate residual
    residual = 0.
    if H<L:
        residual = np.sum(np.sum(Y**2)-np.sum(s**2))

    #Estimation of the variance when sigma2 is unspecified
    if sigma2 is None: 
        #xubar = (1+tauubar)*(1+alpha/tauubar)
        xubar = 1+alpha+(np.sqrt(alpha)*(tauubar+1/tauubar))
        
        eH_ub = int(np.min([np.ceil(L/(1+alpha))-1, H]))-1
        
        upper_bound = (np.sum(s**2)+residual)/(L*M)
        #lower_bound = np.max([(s[eH_ub+1]**2)/(M*xubar), np.mean(s[eH_ub+1:]**2)/M])
        #lower_bound = np.max([(s[eH_ub+1]**2)/(M*xubar), np.sum(s[eH_ub+1:]**2)/(M*(L-eH_ub*(1+alpha)))])
        lower_bound = np.min([(s[eH_ub+1]**2)/(M*xubar), np.sum(s[eH_ub+1:]**2)/(M*(L-eH_ub*(1+alpha)))])

        scale = 1.#/lower_bound
        s = s*np.sqrt(scale)
        residual = residual*scale
        lower_bound = lower_bound*scale
        upper_bound = upper_bound*scale

        sigma2_opt = minimize_scalar(EVBsigma2, args=(L,M,s,residual,xubar), bounds=[lower_bound, upper_bound], method='Bounded')
        sigma2 = sigma2_opt.x

    #Threshold gamma term
    #threshold = np.sqrt(M*sigma2*(1+tauubar)*(1+alpha/tauubar))
    #threshold = np.sqrt(sigma2*(M+L+np.sqrt(L*M)*(tauubar+1/tauubar)))
    threshold = np.sqrt(sigma2*(M+L+np.sqrt(L*M)*(tauubar+1/tauubar)))/np.sqrt(M*sigma2)
    
    pos = np.sum(s>threshold)
    
    #if pos == 0:
        #pos = int(eH_ub/2)
        
    #if pos > eH_ub:
        #pos = eH_ub
    

    #Formula (15) from [2]
    d = np.multiply(s[:pos]/2, 1-np.divide((L+M)*sigma2, s[:pos]**2) + np.sqrt((1-np.divide((L+M)*sigma2, s[:pos]**2))**2 -4*L*M*sigma2**2/s[:pos]**4))

    return np.diag(d)

In [25]:
def EVBsigma2(sigma2,L,M,s,residual,xubar):
    H = len(s)

    alpha = L/M
    x = s**2/(M*sigma2) 

    z1 = x[x>xubar]
    z2 = x[x<=xubar]
    tau_z1 = tau(z1, alpha)

    term1 = np.sum(z2 - np.log(z2))
    term2 = np.sum(z1 - tau_z1)
    term3 = np.sum( np.log( np.divide(tau_z1+1, z1)))
    term4 = alpha*np.sum(np.log(tau_z1/alpha+1))
    
    obj = term1+term2+term3+term4+ residual/(M*sigma2) + (L-H)*np.log(sigma2)

    return obj

In [26]:
def estimate_ranks(layer):
    """ Unfold the 2 modes of the Tensor the decomposition will 
    be performed on, and estimates the ranks of the matrices using VBMF 
    """

    weights = tl.tensor(layer.weight.data)
    unfold_0 = tl.base.unfold(weights, 0) 
    unfold_1 = tl.base.unfold(weights, 1)
    diag_0 = Simple_EVBMF(unfold_0)
    diag_1 = Simple_EVBMF(unfold_1)
    ranks = [diag_0.shape[0], diag_1.shape[1]]
    return ranks

In [89]:
def cp_decomposition_conv_layer(layer, rank):
    """ Gets a conv layer and a target rank, 
        returns a nn.Sequential object with the decomposition """

    # Perform CP decomposition on the layer weight tensorly. 
    last, first, vertical, horizontal = \
        parafac(layer.weight.data, rank=rank, init='svd')

    pointwise_s_to_r_layer = torch.nn.Conv2d(in_channels=first.shape[0], \
            out_channels=first.shape[1], kernel_size=1, stride=1, padding=0, 
            dilation=layer.dilation, bias=False)

    depthwise_vertical_layer = torch.nn.Conv2d(in_channels=vertical.shape[1], 
            out_channels=vertical.shape[1], kernel_size=(vertical.shape[0], 1),
            stride=1, padding=(layer.padding[0], 0), dilation=layer.dilation,
            groups=vertical.shape[1], bias=False)

    depthwise_horizontal_layer = \
        torch.nn.Conv2d(in_channels=horizontal.shape[1], \
            out_channels=horizontal.shape[1], 
            kernel_size=(1, horizontal.shape[0]), stride=layer.stride,
            padding=(0, layer.padding[0]), 
            dilation=layer.dilation, groups=horizontal.shape[1], bias=False)

    pointwise_r_to_t_layer = torch.nn.Conv2d(in_channels=last.shape[1], \
            out_channels=last.shape[0], kernel_size=1, stride=1,
            padding=0, dilation=layer.dilation, bias=True)

    pointwise_r_to_t_layer.bias.data = layer.bias.data

    depthwise_horizontal_layer.weight.data = \
        torch.transpose(horizontal, 1, 0).unsqueeze(1).unsqueeze(1)
    depthwise_vertical_layer.weight.data = \
        torch.transpose(vertical, 1, 0).unsqueeze(1).unsqueeze(-1)
    pointwise_s_to_r_layer.weight.data = \
        torch.transpose(first, 1, 0).unsqueeze(-1).unsqueeze(-1)
    pointwise_r_to_t_layer.weight.data = last.unsqueeze(-1).unsqueeze(-1)

    new_layers = [pointwise_s_to_r_layer, depthwise_vertical_layer, \
                    depthwise_horizontal_layer, pointwise_r_to_t_layer]
    
    return nn.Sequential(*new_layers)

In [22]:
def tucker_decomposition_conv_layer(layer):
    """ Gets a conv layer, 
        returns a nn.Sequential object with the Tucker decomposition.
        The ranks are estimated with a Python implementation of VBMF
        https://github.com/CasvandenBogaard/VBMF
    """

    ranks = estimate_ranks(layer)
    print(layer, "VBMF Estimated ranks", ranks)
    core, [last, first] = \
        partial_tucker(tl.tensor(layer.weight.data), \
            modes=[0, 1], n_iter_max=2000000, rank=ranks, init='svd')
    
    #if layer.groups > 1:
        #first = first.repeat(layer.groups,0)
    

    # A pointwise convolution that reduces the channels from S to R3
    first_layer = torch.nn.Conv2d(in_channels=first.shape[0], \
            out_channels=first.shape[1], kernel_size=1,
            stride=1, padding=0, dilation=layer.dilation, bias=False)

    # A regular 2D convolution layer with R3 input channels 
    # and R3 output channels
    core_layer = torch.nn.Conv2d(in_channels=core.shape[1], \
            out_channels=core.shape[0], kernel_size=layer.kernel_size,
            stride=layer.stride, padding=layer.padding, dilation=layer.dilation,
            bias=False)

    # A pointwise convolution that increases the channels from R4 to T
    last_layer = torch.nn.Conv2d(in_channels=last.shape[1], \
        out_channels=last.shape[0], kernel_size=1, stride=1,
        padding=0, dilation=layer.dilation, bias=True)
    
    if hasattr(layer, "bias") and layer.bias is not None:
          last_layer.bias.data = layer.bias.data

    first = torch.from_numpy(first.copy())
    last = torch.from_numpy(last.copy())
    core = torch.from_numpy(core.copy())
    
    first_layer.weight.data = \
        torch.transpose(first, 1, 0).unsqueeze(-1).unsqueeze(-1)
    last_layer.weight.data = last.unsqueeze(-1).unsqueeze(-1)
    core_layer.weight.data = core

    new_layers = [first_layer, core_layer, last_layer]
    return nn.Sequential(*new_layers)

## tensor decomposition

In [8]:
#load base model

model_config_path = "exp/0.5177_100epoch_1120/model.yml"
data_config_path = "exp/0.5177_100epoch_1120/data.yml"

model_config = read_yaml(cfg=model_config_path)
data_config = read_yaml(cfg=data_config_path)

In [9]:
model_instance = Model(model_config, verbose=True)

idx |   n |     params |          module |            arguments |   in_channel |   out_channel
----------------------------------------------------------------------------------------------
  0 |   3 |        816 |          DWConv | [16, 3, 2, None, 'ReLU'] |            3           16
  1 |   1 |      2,016 | InvertedResidualv2 |           [32, 2, 2] |           16           32
  2 |   4 |      2,288 | InvertedResidualv2 |           [16, 1, 2] |           32           16
  3 |   5 |      7,360 | InvertedResidualv2 |           [16, 2, 2] |           16           16
  4 |   2 |    240,656 | InvertedResidualv3 | [5, 3.5, 128, 1, 1, 2] |           16          128
  5 |   1 |     83,200 |            Conv |          [640, 1, 1] |          128          640
  6 |   1 |          0 |   GlobalAvgPool |                   [] |          640          640
  7 |   1 |      5,778 |       FixedConv | [9, 1, 1, None, 1, None] |          640            9
Model Summary: 161 layers, 342,114 parameters, 342,1

In [17]:
#log path

log_dir = os.path.join("exp", datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
os.makedirs(log_dir, exist_ok=True)

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
#load best weight

model_path = 'exp/0.6420_150epoch_5800/best.pt'

if os.path.isfile(model_path):
    model_instance.model.load_state_dict(torch.load(model_path, map_location=device))

In [10]:
#calculate original model macs

macs = calc_macs(model_instance.model, (3, data_config["IMG_SIZE"], data_config["IMG_SIZE"]))
print(f"macs: {macs}")

macs: 11242418.0


In [46]:
for i,module in enumerate(model_instance.model._modules.values()):
    if i == 3:
        print(module)
        break

Sequential(
  (0): InvertedResidualv3(
    (conv): Sequential(
      (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): HardSwish()
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64, bias=False)
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): SqueezeExcitation(
        (fc1): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
        (relu): ReLU(inplace=True)
        (fc2): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
        (hardsigmoid): HardSigmoid()
      )
      (6): HardSwish()
      (7): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (8): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
)


In [47]:
tucker_decomposition_conv_layer(module[0].conv[3])

[9.765853   5.261058   5.0744658  3.3702116  1.5919446  1.3953285
 0.9876509  0.74641436 0.6215681 ]
7
4
[12.9057865]
0


IndexError: index 1 is out of bounds for axis 0 with size 1

In [68]:
module.conv = tucker_decomposition_conv_layer(module.conv)

Conv2d(128, 9, kernel_size=(1, 1), stride=(1, 1), padding=[0], bias=False) VBMF Estimated ranks [1, 1]


In [142]:
torch.save(model_instance.model,'exp/2021-06-05_18-50-37/decomposed_model.pt')

In [ ]:
#calculate decomposition model

macs = calc_macs(model_instance.model, (3, data_config["IMG_SIZE"], data_config["IMG_SIZE"]))
print(f"macs: {macs}")

## inference decomposition model

In [149]:
CLASSES = ['Battery', 'Clothing', 'Glass', 'Metal', 'Paper', 'Paperpack', 'Plastic', 'Plasticbag', 'Styrofoam']

class CustomImageFolder(ImageFolder):
    """ImageFolder with filename."""

    def __getitem__(self, index):
        img_gt = super(CustomImageFolder, self).__getitem__(index)
        fdir = self.imgs[index][0]
        fname = fdir.rsplit(os.path.sep, 1)[-1]
        return (img_gt + (fname,))

def get_dataloader(img_root: str, data_config: str) -> DataLoader:
    """Get dataloader.

    Note:
	Don't forget to set normalization.
    """
    # Load yaml
    data_config = read_yaml(data_config)

    transform_test_args = data_confg["AUG_TEST_PARAMS"] if data_config.get("AUG_TEST_PARAMS") else None
    # Transformation for test
    transform_test = getattr(
        __import__("src.augmentation.policies", fromlist=[""]),
        data_config["AUG_TEST"],
    )(dataset=data_config["DATASET"], img_size=data_config["IMG_SIZE"])

    dataset = CustomImageFolder(root=img_root, transform=transform_test)
    dataloader = DataLoader(
	dataset=dataset,
	batch_size=1,
	num_workers=8
    )
    return dataloader

In [151]:
@torch.no_grad()
def inference(model, dataloader, dst_path: str):
    result = {}
    model = model.to(device)
    model.eval()
    submission_csv = {}
    for img, _, fname in dataloader:
        img = img.to(device)
        pred, enc_data = run_model(model, img)
        pred = torch.argmax(pred)
        submission_csv[fname[0]] = CLASSES[int(pred.detach())]

    result["macs"] = enc_data
    result["submission"] = submission_csv
    j = json.dumps(result, indent=4)
    save_path = os.path.join(dst_path, 'submission.csv')
    with open(save_path, 'w') as outfile:
        json.dump(result, outfile)

In [491]:
#prepare path

dst='/opt/ml/'
data_config = 'exp/0.6420_150epoch_5800/data.yml'
img_root = 'input/data/test'

In [490]:
model_instance.model = model_instance.model.to('cpu')

In [486]:
# prepare datalaoder
dataloader = get_dataloader(img_root=img_root, data_config=data_config)

# inference
inference(model_instance.model, dataloader, dst)

## fine tuning decomposition model

In [71]:
log_dir = 'exp/2021-06-05_18-50-37'

In [72]:
log_dir

'exp/2021-06-05_18-50-37'

In [493]:
model_config = read_yaml(cfg="exp/0.6420_150epoch_5800/model.yml")
data_config = read_yaml(cfg="exp/0.6420_150epoch_5800/data.yml")

model_config = read_yaml(cfg=model_config)
data_config = read_yaml(cfg=data_config)

In [ ]:
train_dl, val_dl, test_dl = create_dataloader(data_config)

In [109]:
train_path = os.path.join(data_config["DATA_PATH"], "train")
model_path = os.path.join(log_dir, "best.pt")

# Create optimizer, scheduler, criterion
optimizer = torch.optim.Adam(
    model_instance.model.parameters(), lr=0.00001
)

scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer=optimizer,
    max_lr=data_config["INIT_LR"],
    steps_per_epoch=len(train_dl),
    epochs=data_config["EPOCHS"],
    pct_start=0.05,
)

criterion = CustomCriterion(
    samples_per_cls=get_label_counts(train_path)
    if data_config["DATASET"] == "TACO"
    else None,
    device=device,
    #loss_type="weighted"
    #loss_type="customloss"
)

# Amp loss scaler
scaler = (
    torch.cuda.amp.GradScaler() if data_config['FP16'] and device != torch.device("cpu") else None
)

In [111]:
# Create trainer
trainer = TorchTrainer(
    model=model_instance.model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    scaler=scaler,
    device=device,
    model_path=model_path,
    verbose=1,
)

In [101]:
model_instance.model = torch.nn.DataParallel(model_instance.model) #half tensor를 float tensor로 바꿔줌

In [112]:
best_acc, best_f1 = trainer.train(
        train_dataloader=train_dl,
        n_epoch=data_config["EPOCHS"],
        val_dataloader=val_dl if val_dl else test_dl,
    )


Model saved. Current best test f1: 0.086



Model saved. Current best test f1: 0.094



Model saved. Current best test f1: 0.103


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3307a26310>
Traceback (most recent call last):
  File "/miniconda/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1101, in __del__
    self._shutdown_workers()
  File "/miniconda/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1075, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/miniconda/lib/python3.8/multiprocessing/process.py", line 147, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3307a26310>
Traceback (most recent call last):
  File "/miniconda/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1101, in __del__
can only join a child process    
self._shutdown_workers()
  File "/miniconda/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1075, in _shutdown_workers
    w.join


Model saved. Current best test f1: 0.103


...


Model saved. Current best test f1: 0.189



Model saved. Current best test f1: 0.190



Model saved. Current best test f1: 0.192



Model saved. Current best test f1: 0.193


KeyboardInterrupt: 